# Defining an ODE system

The starting point, when using PyGOM to study an ODE system, is to encapsulate the relevant information into a class {class}`.SimulateOde`.
Once defined as an instance of this class, the ODE system is ready for the application of PyGOM's various features such as simulation and parameter fitting.

## Equations vs Transitions

Firstly, we outline the two main ways we might think about defining our ODE system.
Typically, we may already know the set of equations which describes the rates of change of the dependent variables:

$$\frac{\mathrm{d} \mathbf{y}}{\mathrm{d} t} = f(\mathbf{y},\boldsymbol{\theta}, t)$$

where $\mathbf{y} = \left(y_{1},\ldots,y_{n}\right)$ is a vector of the dependent variables, $\boldsymbol{\theta} = \left(\theta_{1},\ldots, \theta_{p}\right)$ is the set of parameters and $f$ is a vector function giving the rates of change of each dependent variable.
Typically, as we have used here, rates of change are with respect to time, $t$, though other variables such as position are also common.

Compartmental models, whereby the dependent variables represent categories or states, have another interpretation in which we may instead consider transitions between the different groups:

$$\begin{aligned}
y_i \rightarrow y_j = f_{i,j}(\mathbf{y},\boldsymbol{\theta}) \\
\end{aligned}$$

where $i, j = \{1,\ldots, n\}$ and $f_{i,j}$ are functions governing the rate at which members of group $i$ transition to group $j$.
Since flows are symmetric ($f_{i,j}=-f_{j,i}$) and a group cannot flow into itself ($f_{i,i}=0$), we need only specify rates for one half of the transition matrix ($i>j$ or $j>i$).

PyGOM allows the user flexibility in choosing which of these perspectives to use, or indeed combine, in order to build their models.
If given transitions, PyGOM will automatically convert these into equations and, {doc}`as we'll see later <unroll/unrollSimple>`, it can also attempt to reverse engineer transitions from equations.

## Example: SIR model

Here we use a Susceptible-Infected-Recovered epidemic model (an SIR model, for short) to demonstrate the two different ways PyGOM supports model specification.
Since we will frequently rely on the SIR model in examples throughout this tutorial, we take a moment here to outline its key features.

```{warning}
There is some ambiguity in the naming of the Recovered class which is also commonly referred to as Removed.
In the latter sense, there is no distinction made between those who can no longer be infected due to infection acquired immunity and infection induced death.
However, for more complex models, the recovered class may become susceptible again due to the effects of immune waning and deaths versus recoveries are typically important to distinguish in real world applications.
Therefore, in this tutorial, the Recovered class will be reserved solely for those who survive infection.
```

The assumptions of the SIR model can be stated as:
1) An average member of the population makes contact sufficient to transmit or receive infection with $c$ others per unit time. Each of these events carries a probability, $p$, of transmission such that each individual has an average of $cp = \beta$ infectious contacts per unit time. This fixed contact rate reflects what is referred to as *standard* incidence, as opposed to *mass-action* incidence, where contacts per person are proportional to the total population size, $N$.
2) The population interacts heterogeneously as if a well mixed continuum.
For instance, a susceptible does not have contacts with other individuals, but with the entire population on average.
3) The infective class recovers at a rate, $\gamma$.
4) No births, deaths (natural or from disease) or migration mean there is no entry into or departure from the population: $S(t)+I(t)+R(t)=N$.

Under these assumptions, the rates of change of the population in each compartment (**S**usceptible, **I**nfected and **R**ecovered) are given by the following equations:

$$\begin{aligned}
\frac{\mathrm{d} S}{\mathrm{d} t} &= -\frac{\beta SI}{N} \\
\frac{\mathrm{d} I}{\mathrm{d} t} &= \frac{\beta SI}{N} - \gamma I \\
\frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma I.
\end{aligned}$$

However, it is equally valid to view this system as the result of two transitions; infection, which takes subjects from susceptible to infected and recovery, which takes them from infected to recovered:

$$\begin{aligned}
S \rightarrow I &= \frac{\beta SI}{N} \\
I \rightarrow R &= \gamma I
\end{aligned}$$

where the left hand side of each equation indicates the transition and the right hand side the rate at which it occurs.
One advantage of specifying our system using the transition approach is that many will find transitions a more intuitive language when thinking about compartmental models.
Perhaps more beneficial, though, is that when building models with transitions using PyGOM, it enables the computer to do our book-keeping when converting transitions to ODE equations.
This reduces the error of, for example, including a flow out of one state, but forgetting to include it in the recipient state.
For example, in the case above, when indicating infections with $-\frac{\beta SI}{N}$ in the $S$ equation, we had to remember to include the opposite sign in the $I$ equation.

## Defining the model with PyGOM

Defining the system is handled by feeding {class}`.Transition` objects into the central class, {class}`.SimulateOde`.
An instance of the {class}`.Transition` class takes one of four types, which are specified by the `transition_type` argument, letting {class}`.SimulateOde` know which type of information it is being given.
This type could be `Transition` or `ODE`, as we've just been discussing, or one of two other available options, namely `Birth` and `Death` processes.
The {class}`.Transition` class accepts 3 or 4 parameters depending on which type is being defined, as summarised in the below table:

|                 |          Transition             |            Equation               |                  Birth                 |                  Death                 |
|:---------------:|:-------------------------------:|:---------------------------------:|:--------------------------------------:|:--------------------------------------:|
| transition_type | T                               | ODE                               | B                                      | D                                      |
| origin          | State transition is from, $y_i$ | State equation pertains to, $y_i$ | State birth process populates, $y_i$   | State death process depletes, $y_i$    |
| destination     | State transition is to, $y_j$   | n/a                               | n/a                                    | n/a                                    |
| equation        | $f_{i,j}$                       | $\frac{dy_i}{dt}$                 | $\frac{dy_i}{dt}$ due to birth process | $\frac{dy_i}{dt}$ due to death process |

```{note}
Arguably, the state which the birth process populates could be defined as a destination rather than an origin, but this is not the convention adopted here.

Furthermore, the {class}`.Transition` class may have been better named.
One of the four possible input types is also called "transition", which would incorrectly imply that it has a special place within the class, given that it shares its name.
```

Let's see how this is implemented for our example SIR system.
We import the relevant modules and then define our transitions:

In [1]:
from pygom import Transition, TransitionType

# Define SIR model through a list of ODEs
odeList = [
    Transition(transition_type=TransitionType.ODE, origin='S', equation='-beta*S*I/N'),
    Transition(transition_type=TransitionType.ODE, origin='I', equation='beta*S*I/N - gamma*I'),
    Transition(transition_type=TransitionType.ODE, origin='R', equation='gamma*I') 
]

# Define SIR model through a list of transitions
transList = [
    Transition(transition_type=TransitionType.T, origin='S', destination='I', equation='beta*S*I/N'),
    Transition(transition_type=TransitionType.T, origin='I', destination='R', equation='gamma*I')
]

We now initialise two {class}`.SimulateOde` objects using these different approaches.
In addition to the ODE or transition information, we must specify which variables are parameters and which refer to states:

In [2]:
stateList = ['S', 'I', 'R']
paramList = ['beta', 'gamma', 'N']

We import the relevant module and define the model in two different ways:

In [3]:
from pygom import SimulateOde
model_ode = SimulateOde(state=stateList, param=paramList, ode=odeList)           # model defined via equations
model_tra = SimulateOde(state=stateList, param=paramList, transition=transList)  # model defined via transitions

We can use the {func}`.get_ode_eqn` function to verify that the models are equivalent:

In [4]:
model_ode.get_ode_eqn()

Matrix([
[          -I*S*beta/N],
[-I*gamma + I*S*beta/N],
[              I*gamma]])

In [5]:
model_tra.get_ode_eqn()

Matrix([
[          -I*S*beta/N],
[-I*gamma + I*S*beta/N],
[              I*gamma]])

## Modifying models

Once an instance of {class}`.SimulateOde` has been defined it may be modified.
For example, let's say we wish to add birth and death processes to the the previously defined SIR model of `model_ode`.
Time dependent parameters can be complicated in PyGOM, so here we will assume that birth and death rates cancel out so that $N$ remains a constant parameter.
If we assume that the birth rate is proportional to the total population, $\mu N$, then balance can be achieved with a death rate from each compartment also proportional to $\mu$ giving a total death rate of $\mu (S+I+R)=\mu N$
We must update the parameters of the class to reflect any additions:

In [6]:
model_ode.param_list = model_ode.param_list + ['mu']

We then include the additional processes, where we assume that all new births enter the population as susceptibles:

In [7]:
birthDeathList = [Transition(origin='S', equation='mu*N', transition_type=TransitionType.B),  
                  Transition(origin='S', equation='mu*S', transition_type=TransitionType.D), 
                  Transition(origin='I', equation='mu*I', transition_type=TransitionType.D), 
                  Transition(origin='R', equation='mu*R', transition_type=TransitionType.D)]

model_ode.birth_death_list = birthDeathList

We can again use {func}`.get_ode_eqn` to verify that the equations are updated correctly:

In [8]:
model_ode.get_ode_eqn()

Matrix([
[   -I*S*beta/N + N*mu - S*mu],
[-I*gamma - I*mu + I*S*beta/N],
[              I*gamma - R*mu]])